In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

### 필요한 모듈 로딩

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM을 통한 요리 정보 해석
- 이미지 -> 맛과 풍미 (image to text)
- 입력 : 요리 이미지(url)
- 출력: 요리명, 요리에 대한 풍미 설명
- 함수로 정의한 다음, RunnableLambda 객체

In [3]:
# 함수 정의 : 이미지 -> 요리명, 풍미 설명 출력
def describe_dish_flavor(input_data):

    prompt = ChatPromptTemplate([
        ("system", """
        You are a culinary expert.
        Based on the input image of a dish,

        you identify the most likely name of the dish by analyzing visual cues such as ingredients, cooking methods, and plating,

        and infer the ingredients and preparation techniques to describe the flavor profile objectively.

        If the dish name is an estimate, this should be stated clearly.
        The flavor description should focus concisely on key taste components (sweetness, saltiness, umami, richness, spiciness, acidity),
        aroma, and textural contrast.
        Subjective judgments and emotional expressions must be excluded, and the explanation should be grounded in observable facts.
        """),
        HumanMessagePromptTemplate.from_template([
            {"text": """아래 이미지의 요리에 대한 요리명과 풍미를 설명해주세요.
             출력형태 :
             요리명 :
             요리의 풍미:
            """},
            {"image_url": "{image_url}"} # image_url는 정해줘 있음.
        ])
    ])

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.1,
        api_key=GOOGLE_API_KEY
    )

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser

    return chain

In [ ]:
from langchain_core.runnables import RunnableLambda

# 함수를 전달인자로 넣기
r1 = RunnableLambda(describe_dish_flavor)

# RunnableLambda를 통한 함수 실행
input_data = {
    "image_url": "https://static.wtable.co.kr/image/production/service/recipe/2068/8e90b171-4b7c-44da-affc-bb9bc8297084.jpg?size=800x800"
    }
# res = r1.invoke(input_data)

In [5]:
print(res)

요리명 : 궁중 떡볶이 (Gungjung Tteokbokki) 또는 간장 떡볶이 (Ganjang Tteokbokki) (추정)

요리의 풍미:
이 요리는 간장 기반의 양념으로 인해 짭조름하면서도 은은한 단맛과 깊은 감칠맛이 특징입니다. 소고기의 고소함과 표고버섯의 흙내음 및 진한 감칠맛이 조화를 이루며, 참기름과 깨가 더해져 고소한 향과 풍미를 더합니다. 식감은 쫄깃한 떡볶이 떡, 부드러운 소고기, 그리고 아삭한 당근의 대비로 이루어져 있습니다. 마늘과 파의 향긋함이 전체적인 풍미를 균형 있게 잡아줍니다. 매운맛은 거의 없으며, 전반적으로 풍부하고 조화로운 맛을 냅니다.


## 2.  요리에 가장 잘 어울리는 wine top 5 검색
- pinecone 백터 db 저장되어 있음
- index : wine-reviews, namespace: wine-reviews-ns

In [6]:
res = """
요리명 : 궁중 떡볶이 (Gungjung Tteokbokki) 또는 간장 떡볶이 (Ganjang Tteokbokki) (추정)
요리의 풍미:
이 요리는 간장 기반의 양념으로 인해 짭조름하면서도 은은한 단맛과 깊은 감칠맛이 특징입니다. 소고기의 고소함과 표고버섯의 흙내음 및 진한 감칠맛이 조화를 이루며, 참기름과 깨가 더해져 고소한 향과 풍미를 더합니다. 식감은 쫄깃한 떡볶이 떡, 부드러운 소고기, 그리고 아삭한 당근의 대비로 이루어져 있습니다. 마늘과 파의 향긋함이 전체적인 풍미를 균형 있게 잡아줍니다. 매운맛은 거의 없으며, 전반적으로 풍부하고 조화로운 맛을 냅니다.
"""

In [ ]:

# 사용자 프롬프트 vector화, 유사도 높은 top=5 찾기
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


# 요리에 대한 풍미(설명) 들어오면,
# 벡터 db에 인덱할 때 사용한 동일 임베딩 모델을 사용해서 임베딩 벡터 생성
embedding = OpenAIEmbeddings(
     model = OPENAI_EMBEDDING_MODEL
 )

# 벡터 db에서 유사도계산, top-5 검색
# 벡터 db 객체 생성
vector_db = PineconeVectorStore(
    embedding = embedding,  # 질문에 대한 임베딩 벡터가 생성됨
    index_name = PINECONE_INDEX_NAME ,
    namespace = PINECONE_NAMESPACE
)
# 벡터 db에서 질문과 가장 유사한, top-5 검색하기
query = res  # 질문
print("질문 : ", query)
print("-"*50)
results = vector_db.similarity_search(query, k=5)  # top-5 검색

질문 :  
요리명 : 궁중 떡볶이 (Gungjung Tteokbokki) 또는 간장 떡볶이 (Ganjang Tteokbokki) (추정)
요리의 풍미:
이 요리는 간장 기반의 양념으로 인해 짭조름하면서도 은은한 단맛과 깊은 감칠맛이 특징입니다. 소고기의 고소함과 표고버섯의 흙내음 및 진한 감칠맛이 조화를 이루며, 참기름과 깨가 더해져 고소한 향과 풍미를 더합니다. 식감은 쫄깃한 떡볶이 떡, 부드러운 소고기, 그리고 아삭한 당근의 대비로 이루어져 있습니다. 마늘과 파의 향긋함이 전체적인 풍미를 균형 있게 잡아줍니다. 매운맛은 거의 없으며, 전반적으로 풍부하고 조화로운 맛을 냅니다.

--------------------------------------------------


In [10]:
# 요리에 가장 잘 어울리는 와인 5개 검색한 결과
results

[Document(id='310a50ab-e2ce-4a6d-97da-9568bc28d960', metadata={'row': 37787.0, 'source': 'c:\\JH\\ggdrive_JH\\kpmg_7th_lab\\rag_pjt_kpmg7th\\prepare\\wine_reviews\\winemag-data-130k-v2.csv'}, page_content=': 37787\ncountry: US\ndescription: Luscious in candied, pastry-filling raspberries and cherries, with dustings of cocoa, cinnamon and vanilla, and a sweet veneer of smoky sandalwood. Could be somewhat crisper and better structured, but a delicious Pinot for drinking now.\ndesignation: Meiomi\npoints: 87\nprice: 25.0\nprovince: California\nregion_1: Sonoma County-Monterey County-Santa Barbara County\nregion_2: California Other\ntaster_name: \ntaster_twitter_handle: \ntitle: Belle Glos 2008 Meiomi Pinot Noir (Sonoma County-Monterey County-Santa Barbara County)\nvariety: Pinot Noir\nwinery: Belle Glos'),
 Document(id='51838d0c-eea3-4992-a2ea-ec3b1d533b00', metadata={'row': 81971.0, 'source': 'c:\\JH\\ggdrive_JH\\kpmg_7th_lab\\rag_pjt_kpmg7th\\prepare\\wine_reviews\\winemag-data-130k-v2.

In [12]:
# "\n".join([doc.page_content for doc in results])

context = ""
for doc in results:
    # print(doc.page_content)
    context = context + doc.page_content + "\n"
    # print("-" * 50)
print(context)

: 37787
country: US
description: Luscious in candied, pastry-filling raspberries and cherries, with dustings of cocoa, cinnamon and vanilla, and a sweet veneer of smoky sandalwood. Could be somewhat crisper and better structured, but a delicious Pinot for drinking now.
designation: Meiomi
points: 87
price: 25.0
province: California
region_1: Sonoma County-Monterey County-Santa Barbara County
region_2: California Other
taster_name: 
taster_twitter_handle: 
title: Belle Glos 2008 Meiomi Pinot Noir (Sonoma County-Monterey County-Santa Barbara County)
variety: Pinot Noir
winery: Belle Glos
: 81971
country: Argentina
description: This almost translucent brick-colored Pinot offers aromas of lettuce leaf, orange peel and dried red fruits. It's light, edgy and fresh in feel, with flavors of salted meat and herbal plum. A tangy citric finish is primed with racy acidity. Fermented and aged in egg-shaped cement tanks.
designation: Eggo Filoso
points: 89
price: 30.0
province: Mendoza Province
regi

In [13]:
context = "\n".join([doc.page_content for doc in results])
print(context)

: 37787
country: US
description: Luscious in candied, pastry-filling raspberries and cherries, with dustings of cocoa, cinnamon and vanilla, and a sweet veneer of smoky sandalwood. Could be somewhat crisper and better structured, but a delicious Pinot for drinking now.
designation: Meiomi
points: 87
price: 25.0
province: California
region_1: Sonoma County-Monterey County-Santa Barbara County
region_2: California Other
taster_name: 
taster_twitter_handle: 
title: Belle Glos 2008 Meiomi Pinot Noir (Sonoma County-Monterey County-Santa Barbara County)
variety: Pinot Noir
winery: Belle Glos
: 81971
country: Argentina
description: This almost translucent brick-colored Pinot offers aromas of lettuce leaf, orange peel and dried red fruits. It's light, edgy and fresh in feel, with flavors of salted meat and herbal plum. A tangy citric finish is primed with racy acidity. Fermented and aged in egg-shaped cement tanks.
designation: Eggo Filoso
points: 89
price: 30.0
province: Mendoza Province
regi

In [14]:
# 요리에 어울리는 와인 top-5 검색결과를 리턴하는 함수 정의
def search_wines(query):
    embedding = OpenAIEmbeddings(
         model = OPENAI_EMBEDDING_MODEL
    )
    
    # 벡터 db에서 유사도계산, top-5 검색
    # 벡터 db 객체 생성
    vector_db = PineconeVectorStore(
        embedding = embedding,  # 질문에 대한 임베딩 벡터가 생성됨
        index_name = PINECONE_INDEX_NAME ,
        namespace = PINECONE_NAMESPACE
    )
    # 벡터 db에서 질문과 가장 유사한, top-5 검색하기

    print("질문 : ", query)
    print("-"*50)
    results = vector_db.similarity_search(query, k=5)  # top-5 검색

    print(results)


In [15]:
# 요리에 어울리는 와인 top-5 검색결과를 리턴하는 함수 호출
query = res  # 질문
def search_wines(query):

SyntaxError: incomplete input (1879863807.py, line 3)

In [ ]:
# 1 사용자는 요리 이미지 업로딩